# WSGI接口

## Web应用的本质

- 浏览器发送一个HTTP请求
- 服务器收到请求，生成一个HTML文档
- 服务器把HTML文档作为HTTP响应的Body发送给浏览器
- 浏览器收到HTTP响应，从HTTP Body取出HTML文档并显示

## 最简单的Web应用

- 先把HTML用文件保存，用一个现成的HTTP服务器软件接收用户请求，从文件中读取HTML，返回。Apache、Nginx、Lighttpd等这些常见的静态服务器就是干这件事的。

## 动态生成HTML

- 需要把上述步骤自己来实现，接受HTTP请求、解析HTTP请求、发送HTTP响应都是苦力活，如果我们自己来写这些底层代码，还没开始写动态HTML呢，就需要花几个月去读HTTP规范。

## 正确的做法：使用WSGI接口

- 底层代码由专门的服务器软件实现，我们用Python专注于生成HTML文档，因为我们不希望接触到TCP连接、HTTP原始请求和响应格式，所以需要一个统一的接口，让我们专心用Python编写web业务。
- 这个接口就是WSGI：Web Server Gateway Interface
- WSGI接口定义非常简单，它只要求Web开发者实现一个函数，就可以响应HTTP请求。

## 最简单的Web版本的“Hello World!”

In [2]:
def application(environ, start_response):
    start_response('200 OK', [('Content-Type', 'text/html')])
    return [b'<h1>Hello, web!</h1>']

- 上面的application()函数就是符合WSGI标准的一个HTTP处理函数，它接收两个参数：
    - environ： 一个包含所有HTTP请求信息的dict对象
    - start_response: 一个发送HTTP响应的函数

- 在application()函数中，调用：

```python
start_response('200 OK', [('Content-Type', 'text/html')])
```

- 就发送了HTTP响应的Header，注意Header只能发送一次，也就是只能调用一次start_response()函数，start_response()函数接收两个参数，一个是HTTP响应码，一个是一组list表示的HTTP Header， 每个Header用一个包含两个str和tuple表示。

- 通常情况下，都应该把Content-Type头发送给浏览器。其他很多常用的HTTP Header也应该发送
- 然后函数的返回值```‘b<h1>Hello web!</h1>'```将作为HTTP响应的Body发送给浏览器

## WSGI的好处

- 有了WSGI，我们关心的就是如何从environ这个dict对象拿到HTTP请求信息，然后构造HTML，通过start_response()发送Header，最后返回Body。
- 整个application()函数本身没有涉及到任何解析HTTP的部分，也就是说底层代码不需要我们自己编写，我们只负责在更高层次上考虑如何响应请求就可以了。
- appllication()必须由WSGI服务器来调用。有很多符合WSGI规范的服务器，我们可以挑选一个采用，我们在这里挑选一个最简单的WSGI把我们的Web应用程序跑起来。

- python内置了一个WSGI服务器，这个模块叫wsgiref，它是用纯Python编写的WSGI服务器的参考实现。所谓“参考实现”是指，该实现完全符合WSGI标准，但是不考虑任何运行效率，仅供开发和测试使用。

## 运行WSGI服务

- 先编写hello.py，实现Web应用程序的WSGI处理函数：

In [4]:
%%writefile hello.py
def application(environ, start_response):
    start_response('200 OK', [('Content-Type', 'text/html')])
    return [b'<h1>Hello, web!</h1>']

Writing hello.py


- 然后，编写一个server.py，负责启动WSGI服务器，加载application()函数：

In [6]:
%%writefile server.py
from wsgiref.simple_server import make_server
from hello import application

# 创建一个服务器，IP地址为空，端口是8000，处理函数是application:
httpd = make_server('', 8000, application)
print('Serving HTTP on port 8000...')
# 开始监听HTTP请求
httpd.serve_forever()

Overwriting server.py


![](1.png)

- environ里读取PATH_INFO，这样可以显示更加动态的内容：

In [10]:
%%writefile hello.py
def application(environ, start_response):
    start_response('200 OK', [('Content-Type', 'text/html')])
    body = '<h1>Hello, {}'.format(environ['PATH_INFO'][1:] or 'web')
    return [body.encode('utf-8')]

Overwriting hello.py


![](2.png)

## 总结

- 无论多么复杂的Web应用程序，入口都是一个WSGI处理函数。HTTP请求的所有输入信息都可以通过environ获得，HTTP响应的输出都可以通过start_response()加上函数的返回值作为Body。
- 复杂的Web应用程序，光靠一个WSGI函数来处理还是显得太底层了，我们需要在WSGI上再抽象出Web框架，进一步简化Web开发。